In [ ]:
from IPython.display import HTML
HTML(filename='/Users/hubatsl/Desktop/Code/Python/src/hide_code.html')

# Test for class DiffusionFitter

In order to test changes made to particle tracking and off rate fitting this document can serve as a standard. Sample data for diffusion contains PAR-6 measurements. Also compare to Matlab's testing file. Both should give roughly the same outputs.

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import os
import time
import sys
# Add path to python modules
sys.path.append(os.path.abspath('..')+'/src/')
from MovieTracks import DiffusionFitter, OffRateFitter, ParameterSampler

In [ ]:
%qtconsole

### Testing tracking, fitting, plotting

First, we test the particle tracking by running on the folder specified in 'fol'. 
After creating an instance of DiffusionFitter (d), d.analyze() is run to find features and link tracks.

In [ ]:
fol = '/Users/hubatsl/Desktop/Code/sample_data/16_07_20_PAR6_2/fov1_16bit/'
d = DiffusionFitter(fol, 300, autoMetaDataExtract=False, featSize=3, link_strat='auto',
                    parallel=True, pixelSize=0.120, saveFigs=False, showFigs=False, timestep=0.033)
t0 = time.time()
d.analyze()
t = time.time() - t0
print('Test took ' + str(t) + ' seconds, normal time ~23 s.')

**Plot calibration of feature finding for one frame (1st frame by default).**

In [ ]:
d.showFigs = True
d.plot_calibration()
if d.features.size == 571230:
    print('Total number of features ' + str(d.features.size) + ', as expected.')
else:
    print('Total number of features ' + str(d.features.size) + ', not as expected 571230.')

**Plot trajectories that are longer than threshold set by user.**

In [ ]:
d.plot_trajectories()
if d.trajectories.particle.unique().size == 117:
    print('Total number of trajectories ' + str(d.trajectories.particle.unique().size) +
          ', as expected.')
else:
    print('Total number of trajectories ' + str(d.trajectories.particle.unique().size) +
          ', not as expected 117.')

**Plot mean square displacement over time.**

In [ ]:
#Plot msd for all tracks
d.plot_msd()
#Plot msd only for part of the tracks
f1, ax = plt.subplots()
plt.plot(d.im.index, d.im.iloc[:, ::10])
plt.xscale('log');
plt.yscale('log');
plt.show()

**Sanity check for individual MSD fits. Splitting into high and low alpha and D.**

In [ ]:
plt.rcParams['figure.figsize'] = (15, 10)
ax1 = plt.subplot(121)
ax1.title.set_text('MSD of high vs low alpha')
ax1.plot(d.im.iloc[:10, ((d.a<1.2)&(d.a>1))], 'b');
ax1.plot(d.im.iloc[:10, ((d.a<0.6)&(d.a>0))], 'r');
plt.xscale('log');
plt.yscale('log');
ax2 = plt.subplot(122)
ax2.title.set_text('MSD of high vs low D')
ax2.plot(d.im.iloc[:10, ((d.D<0.5)&(d.D>0.1))], 'c');
ax2.plot(d.im.iloc[:10, ((d.D<0.1)&(d.D>0))], 'm');
plt.xscale('log');
plt.yscale('log');
plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = (15, 5)
ax1 = plt.subplot(121)
ax1.title.set_text('MSD of high vs low alpha')
ax1.plot(d.im.iloc[:10, ((d.a<1.2)&(d.a>1))].mean(axis=1), 'b');
ax1.plot(d.im.iloc[:10, ((d.a<0.6)&(d.a>0))].mean(axis=1), 'r');
ax2 = plt.subplot(122)
ax2.title.set_text('MSD of high vs low D')
ax2.plot(d.im.iloc[:10, ((d.D<0.5)&(d.D>0.1))].mean(axis=1), 'c');
ax2.plot(d.im.iloc[:10, ((d.D<0.1)&(d.D>0))].mean(axis=1), 'm');
plt.show()

**Sanity check for individual fits, exemplified for high alphas.**

In [ ]:
plt.rcParams['figure.figsize'] = (8, 7)
d_highAlpha=d.im.iloc[:10, ((d.a<1.2)&(d.a>1))]
for c in d_highAlpha.columns:
    res = np.polyfit(np.log10(d_highAlpha.index), np.log10(d_highAlpha[c]), 1, full=True)
    DA = [res[0][0], res[0][1]]
    D = 10**DA[1]/4
    a = DA[0]
    plt.plot(d_highAlpha.index, 4*D*d_highAlpha.index**a, 'b')
plt.plot(d.im.index[:10], d.im.iloc[:10, ((d.a<1.2)&(d.a>1))], 'r');
plt.show()

This shows that there is not the expected difference in average MSD behavior between 'fast' (high D, high $\alpha$) and 'slow' (low D, low $\alpha$) particles.

**Finally, fit $\langle x \rangle = 4Dt^\alpha$ and plot D vs $\alpha$**

In [ ]:
d.plot_diffusion_vs_alpha()
d.D_restricted

In [ ]:
d.plot_diffusion_vs_alpha_verbose()

In [ ]:
if d.D.mean()==0.12002050139512239:
    print('Mean d is ' + str(d.D.mean()) + ', as expected.')
else:
    print('Mean d is ' + str(d.D.mean()) + ', not as expected 0.12002050139512239.')

In [ ]:
part_count = d.trajectories['particle'].value_counts()
n, bins, patches = plt.hist(part_count.asobject, range(80, 500, 10))
plt.show()

**Only analyse frames 101-400, to test startFrame and endFrame.**

In [ ]:
fol = '/Users/hubatsl/Desktop/Code/sample_data/16_07_20_PAR6_2/fov1_16bit/'
d_short = DiffusionFitter(fol, 300, autoMetaDataExtract=False, endFrame=401, featSize=3, link_strat='auto',
                    parallel=True, pixelSize=0.120, saveFigs=False, showFigs=False, startFrame=101,
                    timestep=0.033)
d_short.analyze()
if ((d_short.D_restricted==0.1940520768268752)&
    (len(d_short.trajectories['particle'].unique())==23)):
        print('D_restricted and number of trajectories as expected.')
else: 
    print('D_restricted and number of trajectories not as expected.')

**Testing make_labelled_movie(), output to self.basepath. Can also be used to check startFrame and endFrame behavior from above by comparing the movies created for d_short and d. Careful, both have the same basepath, so will overwrite each other if run consecutively.**

Create movie with one particle centered in the middle, cutting off the frames in the beginning and the end when the particle is not tracked (isolate=True) and for all frames and all particles (isolate=False).

In [ ]:
d_short.make_labelled_movie([d_short.trajectories['particle'].unique()[1]], isolate=True)
d_short.make_labelled_movie(d_short.trajectories['particle'].unique(), isolate=False)

In [ ]:
d.make_labelled_movie([d.trajectories['particle'].unique()[1]], isolate=True)
d.make_labelled_movie(d.trajectories['particle'].unique(), isolate=False)

**Delete tiff files written to disk at beginning of the test.**

In [ ]:
d.delete_images()

### Test whether background particles could be picked up as diffusive species.

**Conclusion:** This really seems to depend on the parameters chosen. For PARs (0.033, no gap, 45% LP) mTL 80 and a threshold that picks up GFP seems to give almost no artefacts. However, dropping minTrackLength to 20 there are a significant number of artefacts which appear at low alphas and Ds.

In [ ]:
bg80 = DiffusionFitter('/Users/hubatsl/Desktop/Code/sample_data/17_02_03/'+
                       'wt_diff_background/1/TIRF_analyse_45p_background.stk',
                    1000, parallel=True, pixelSize=0.120, saveFigs=True, showFigs=False,
                    autoMetaDataExtract=True, startFrame=0, minTrackLength=80)
bg80.plot_calibration(1)
bg80.analyze()

In [ ]:
bg80.showFigs = True
bg80.plot_trajectories()

In [ ]:
bg20 = DiffusionFitter('/Users/hubatsl/Desktop/Code/sample_data/17_02_03/'+
                       'wt_diff_background/1/TIRF_analyse_45p_background.stk',
                    1000, parallel=True, pixelSize=0.120, saveFigs=True, showFigs=False,
                    autoMetaDataExtract=True, startFrame=0, minTrackLength=20)
bg20.plot_calibration(1)
bg20.analyze()

In [ ]:
bg20.showFigs = True
bg20.plot_trajectories()

In [ ]:
bg20.plot_diffusion_vs_alpha()

**The above analysis of the fluorescent particles in the media, outside of the embryo, suggest that these particles are not picked up by the algorithm minTrackLength is chosen sufficiently high. Control embryo with actual membrane particles in the same folder see below:**

In [ ]:
d80 = DiffusionFitter('/Users/hubatsl/Desktop/Code/sample_data/17_02_03/'+
                      'wt_diff_background/1/TIRF_analyse_45p.stk',
                      1000, parallel=True, pixelSize=0.120, saveFigs=True, showFigs=False,
                      autoMetaDataExtract=True, startFrame=0, minTrackLength=80)
d80.plot_calibration(1)
d80.analyze()

In [ ]:
d80.plot_trajectories()

In [ ]:
d80.showFigs = True
d80.plot_diffusion_vs_alpha()

In [ ]:
d20 = DiffusionFitter('/Users/hubatsl/Desktop/Code/sample_data/17_02_03/'+
                      'wt_diff_background/1/TIRF_analyse_45p.stk',
                    1000, parallel=True, pixelSize=0.120, saveFigs=True, showFigs=False,
                    autoMetaDataExtract=True, startFrame=3500, minTrackLength=20)
d20.plot_calibration(1)
d20.analyze()

In [ ]:
d20.plot_trajectories()

In [ ]:
d20.showFigs=True
d20.plot_diffusion_vs_alpha()

In [ ]:
dN2 = DiffusionFitter('/Volumes/Transcend/SPT/Diffusion/N2_background/17_06_05/plate1_N2_25deg/3/100p.stk',
                    2000, minTrackLength=20)
dN2.plot_calibration(1)
dN2.analyze()

In [ ]:
dN2.showFigs=True
dN2.plot_diffusion_vs_alpha()

In [ ]:
dN2 = DiffusionFitter('/Volumes/Transcend/SPT/Diffusion/N2_background/17_06_05/plate1_N2_25deg/1/100p.stk',
                    2000, minTrackLength=20)
dN2.plot_calibration(1)
dN2.analyze()

In [ ]:
dN2.showFigs=True
dN2.plot_trajectories()
dN2.plot_diffusion_vs_alpha()

### Testing ROI filtering and filtering of trajectories by time.

In [ ]:
%matplotlib notebook
d20.def_ROI()

In [ ]:
d20.apply_ROI(useAllFeats=True)
d20.link_feats(useAllFeats=False)

In [ ]:
d20.plot_trajectories()

In [ ]:
d20.filt_traj_by_t(3700, 4000)

In [ ]:
d20.plot_trajectories()